# Action Recognition @ UCF101  
**Due date: 11:59 pm on Nov. 19, 2019 (Tuesday)**

## Description
---
In this homework, you will be doing action recognition using Recurrent Neural Network (RNN), (Long-Short Term Memory) LSTM in particular. You will be given a dataset called UCF101, which consists of 101 different actions/classes and for each action, there will be 145 samples. We tagged each sample into either training or testing. Each sample is supposed to be a short video, but we sampled 25 frames from each videos to reduce the amount of data. Consequently, a training sample is an image tuple that forms a 3D volume with one dimension encoding *temporal correlation* between frames and a label indicating what action it is.

To tackle this problem, we aim to build a neural network that can not only capture spatial information of each frame but also temporal information between frames. Fortunately, you don't have to do this on your own. RNN — a type of neural network designed to deal with time-series data — is right here for you to use. In particular, you will be using LSTM for this task.

Instead of training an end-to-end neural network from scratch whose computation is prohibitively expensive, we divide this into two steps: feature extraction and modelling. Below are the things you need to implement for this homework:
- **{35 pts} Feature extraction**. Use any of the [pre-trained models](https://pytorch.org/docs/stable/torchvision/models.html) to extract features from each frame. Specifically, we recommend not to use the activations of the last layer as the features tend to be task specific towards the end of the network. 
    **hints**: 
    - A good starting point would be to use a pre-trained VGG16 network, we suggest first fully connected layer `torchvision.models.vgg16` (4096 dim) as features of each video frame. This will result into a 4096x25 matrix for each video. 
    - Normalize your images using `torchvision.transforms` 
    ```
    normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    prep = transforms.Compose([ transforms.ToTensor(), normalize ])
    prep(img)
    The mean and std. mentioned above is specific to Imagenet data
    
    ```
    More details of image preprocessing in PyTorch can be found at http://pytorch.org/tutorials/beginner/data_loading_tutorial.html
    
- **{35 pts} Modelling**. With the extracted features, build an LSTM network which takes a **dx25** sample as input (where **d** is the dimension of the extracted feature for each frame), and outputs the action label of that sample.
- **{20 pts} Evaluation**. After training your network, you need to evaluate your model with the testing data by computing the prediction accuracy **(5 points)**. The baseline test accuracy for this data is 75%, and **10 points** out of 20 is for achieving test accuracy greater than the baseline. Moreover, you need to compare **(5 points)** the result of your network with that of support vector machine (SVM) (stacking the **dx25** feature matrix to a long vector and train a SVM).
- **{10 pts} Report**. Details regarding the report can be found in the submission section below.

Notice that the size of the raw images is 256x340, whereas your pre-trained model might take **nxn** images as inputs. To solve this problem, instead of resizing the images which unfavorably changes the spatial ratio, we take a better solution: Cropping five **nxn** images, one at the image center and four at the corners and compute the **d**-dim features for each of them, and average these five **d**-dim feature to get a final feature representation for the raw image.
For example, VGG takes 224x224 images as inputs, so we take the five 224x224 croppings of the image, compute 4096-dim VGG features for each of them, and then take the mean of these five 4096-dim vectors to be the representation of the image.

In order to save you computational time, you need to do the classification task only for **the first 25** classes of the whole dataset. The same applies to those who have access to GPUs. **Bonus 10 points for running and reporting on the entire 101 classes.**


## Dataset
Download **dataset** at [UCF101](http://vision.cs.stonybrook.edu/~yangwang/public/UCF101_images.tar)(Image data for each video) and the **annos folder** which has the video labels and the label to class name mapping is included in the assignment folder uploaded. 


UCF101 dataset contains 101 actions and 13,320 videos in total.  

+ `annos/actions.txt`  
  + lists all the actions (`ApplyEyeMakeup`, .., `YoYo`)   
  
+ `annots/videos_labels_subsets.txt`  
  + lists all the videos (`v_000001`, .., `v_013320`)  
  + labels (`1`, .., `101`)  
  + subsets (`1` for train, `2` for test)  

+ `images/`  
  + each folder represents a video
  + the video/folder name to class mapping can be found using `annots/videos_labels_subsets.txt`, for e.g. `v_000001` belongs to class 1 i.e. `ApplyEyeMakeup`
  + each video folder contains 25 frames  



## Some Tutorials
- Good materials for understanding RNN and LSTM
    - http://blog.echen.me
    - http://karpathy.github.io/2015/05/21/rnn-effectiveness/
    - http://colah.github.io/posts/2015-08-Understanding-LSTMs/
- Implementing RNN and LSTM with PyTorch
    - [LSTM with PyTorch](http://pytorch.org/tutorials/beginner/nlp/sequence_models_tutorial.html#sphx-glr-beginner-nlp-sequence-models-tutorial-py)
    - [RNN with PyTorch](http://pytorch.org/tutorials/intermediate/char_rnn_classification_tutorial.html)

---
---
## **Problem 1.** Feature extraction

In [2]:
# Mount your google drive where you've saved your assignment folder
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [3]:
# Set your working directory (in your google drive)
# Note that 'gdrive/My Drive/Y2019Fall/CSE-527-Intro-To-Computer-Vision/hw4' is just an example, 
#   change it to your specific homework directory.
cd '/content/gdrive/My Drive/CV/Wang_Tianao_112819772_hw5'

/content/gdrive/My Drive/CV/Wang_Tianao_112819772_hw5


In [0]:
#extract images
import tarfile
tar = tarfile.open("UCF101_images.tar", "r:")
tar.extractall(path="/content/gdrive/My Drive/CV/Wang_Tianao_112819772_hw5/images")
tar.close()

In [0]:
# \*write your codes for feature extraction (You can use multiple cells, this is just a place holder)
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
import torchvision
import torchvision.transforms as transforms

import numpy as np
import scipy as sp
import cv2
from scipy.io import savemat
import os

In [6]:
vgg16 = torchvision.models.vgg16(pretrained=True)
del vgg16.classifier[2:]
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
prep = transforms.Compose([ transforms.ToTensor(), normalize ])
print(vgg16)

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1

In [8]:
#read data
f1 = open("./annos/actions.txt","r")
f2 = open("./annos/videos_labels_subsets.txt","r")
classNumber = 25
class_names = []
line = f1.readline()
while line:
    num = list(line.split())
    class_names.append(num[1])
    line = f1.readline()
f1.close()
print(class_names)

dir_names = []
vedio_class_names = []
trainDir = []
trainLabel = []
testDir = []
testLabel = []
line = f2.readline()
while line:
    num = list(line.split())
    if (int(num[1]) <= classNumber):
      dir_names.append(num[0])
      vedio_class_names.append(num[1])

      if (num[2] == "1"):
        trainDir.append(num[0])
        trainLabel.append(num[1])
      else:
        testDir.append(num[0])
        testLabel.append(num[1])
    line = f2.readline()
f2.close()
print(dir_names)
print(vedio_class_names)
print(trainDir)
print(trainLabel)
print(testDir)
print(testLabel)

['ApplyEyeMakeup', 'ApplyLipstick', 'Archery', 'BabyCrawling', 'BalanceBeam', 'BandMarching', 'BaseballPitch', 'Basketball', 'BasketballDunk', 'BenchPress', 'Biking', 'Billiards', 'BlowDryHair', 'BlowingCandles', 'BodyWeightSquats', 'Bowling', 'BoxingPunchingBag', 'BoxingSpeedBag', 'BreastStroke', 'BrushingTeeth', 'CleanAndJerk', 'CliffDiving', 'CricketBowling', 'CricketShot', 'CuttingInKitchen', 'Diving', 'Drumming', 'Fencing', 'FieldHockeyPenalty', 'FloorGymnastics', 'FrisbeeCatch', 'FrontCrawl', 'GolfSwing', 'Haircut', 'Hammering', 'HammerThrow', 'HandstandPushups', 'HandstandWalking', 'HeadMassage', 'HighJump', 'HorseRace', 'HorseRiding', 'HulaHoop', 'IceDancing', 'JavelinThrow', 'JugglingBalls', 'JumpingJack', 'JumpRope', 'Kayaking', 'Knitting', 'LongJump', 'Lunges', 'MilitaryParade', 'Mixing', 'MoppingFloor', 'Nunchucks', 'ParallelBars', 'PizzaTossing', 'PlayingCello', 'PlayingDaf', 'PlayingDhol', 'PlayingFlute', 'PlayingGuitar', 'PlayingPiano', 'PlayingSitar', 'PlayingTabla', 'P

In [0]:
def imgResize(img,size=(224,224)):
  # print(img.shape)
  # list = []
  # list.append(img[:size[0],:size[1]])
  # list.append(img[(img.shape[0]-size[0]):,:size[1]])
  # list.append(img[:size[1],(img.shape[1]-size[1]):])
  # list.append(img[(img.shape[0]-size[0]):,(img.shape[1]-size[1]):])
  # list.append(img[(img.shape[0]-size[0])//2:(img.shape[0]+size[0])//2,(img.shape[1]-size[1])//2:(img.shape[1]+size[1])//2])

  # print(list[0].shape)
  # test = np.zeros((size[0], size[1], 3), dtype=np.int)
  # for i in range(size[0]):
  #   for j in range(size[1]):
  #     im
  # return list
  list =  (img[:, :size[0], :size[1]], img[:, (img.shape[1]-size[0]):, :size[1]], 
      img[:, :size[0], (img.shape[2]-size[1]):],
      img[:, (img.shape[1]-size[0]):, (img.shape[2]-size[1]):], 
      img[:, (img.shape[1]-size[0])//2:(img.shape[1]+size[0])//2, (img.shape[2]-size[1])//2:(img.shape[2]+size[1])//2])

  img = vgg16(torch.stack(list))
  img = img.mean(dim=0)
  return img

In [12]:
for Dir in trainDir:
  features = []
  if not os.path.exists(os.path.join("./mat_feature/", Dir+'.mat')):
    for f in os.listdir(os.path.join("./images/images/", Dir)):
      fname = os.path.join("./images/images/", Dir, f)
      img = prep(cv2.imread(fname))
      with torch.no_grad():
        img = imgResize(img)
        features.append(img)
    savemat(os.path.join("./mat_feature/", Dir+'.mat'), {'Feature':torch.stack(features).detach().numpy()})
print("train extract finish")

for Dir in testDir:
  features = []
  if not os.path.exists(os.path.join("./mat_feature_test/", Dir+'.mat')):
    for f in os.listdir(os.path.join("./images/images/", Dir)):
      fname = os.path.join("./images/images/", Dir, f)
      img = prep(cv2.imread(fname))
      with torch.no_grad():
        img = imgResize(img)
        features.append(img)
    savemat(os.path.join("./mat_feature_test/", Dir+'.mat'), {'Feature':torch.stack(features).detach().numpy()})
print("test extract finish")

train extract finish
test extract finish


***
***
## **Problem 2.** Modelling

* ##### **Print the size of your training and test data**

In [65]:
train = []
test = []

for i in range(len(trainDir)):
  val = torch.tensor((int)(trainLabel[i])-1)
  data = torch.tensor(sp.io.loadmat(os.path.join("./mat_feature/",trainDir[i]+'.mat'))['Feature'])
  train.append((data,val))
  # if i % 100 == 0:
  #   print("train " + str(i) + " finish")
print("Finish load train data")
for i in range(len(testDir)):
  val = torch.tensor((int)(testLabel[i])-1)
  data = torch.tensor(sp.io.loadmat(os.path.join("./mat_feature_test/",testDir[i]+'.mat'))['Feature'])
  test.append((data,val))
  # if i % 100 == 0:
  #   print("test " + str(i) + " finish")
print("Finish load test data")

Finish load train data
Finish load test data


In [66]:
# Don't hardcode the shape of train and test data
print('Length of training data is :', len(train))
print('Shape of each training data is :', train[0][0].shape)
print('Length of test/validation data is :', len(test))
print('Shape of each test/validation data is :', test[0][0].shape)

Length of training data is : 2409
Shape of each training data is : torch.Size([25, 4096])
Length of test/validation data is : 951
Shape of each test/validation data is : torch.Size([25, 4096])


In [0]:
class LSTM(nn.Module):
  def __init__(self,feature_dim,hidden_size,action_size):
    super(LSTM,self).__init__()
    self.hidden_size = hidden_size
    self.lstm = nn.LSTM(feature_dim,hidden_size)
    self.out = nn.Linear(hidden_size,action_size)
    self.hidden = self.init_hidden()
  
  def init_hidden(self):
    return (torch.zeros(1,1,self.hidden_size),torch.zeros(1,1,self.hidden_size))
  
  def forward(self,vedio):
    lstm_out, self.hidden = self.lstm(vedio.view(len(vedio),1,-1),self.hidden)
    tag_space = self.out(self.hidden[0])
    #Try to return tag_scores, but it doesn't work well, accuracy = 4%
    #Change log_softmax() into sigmoid() also doesn't work well, accuracy = 4%
    # tag_scores = F.log_softmax(tag_space, dim=1)
    return tag_space

In [0]:
#Changing parameters shows that hidden_size 128 is better than 64, 64 is better than 32
#hidden_size = 32, accuracy = 48%
#hidden_size = 64, accuracy = 68%
#hidden_size = 128, accuracy = 82%
model = LSTM(4096,128,25)
loss_function = nn.CrossEntropyLoss()

#Adam doesn't work well
#lr = 0.01 is better than 0.1
#lr = 0.1,accuracy = 45%
#lr = 0.01,accuracy = 82%
optimizer = optim.SGD(model.parameters(),lr=0.01)

#Using Adam, accuracy = 4%
#optimizer = optim.Adam(model.parameters(), lr=0.01) 

In [122]:
print(model)

LSTM(
  (lstm): LSTM(4096, 128)
  (out): Linear(in_features=128, out_features=25, bias=True)
)


In [123]:
#epoch number = 20, accuracy = 40%
#epoch number = 40, accuracy = 76%
#epoch number = 60, accuracy = 82%

for epoch in range(60):
  print("epoch"+str(epoch+1))
  running_loss = 0.0
  for vedio,labels in train:
    model.zero_grad()
    model.hidden = model.init_hidden()
    tag_scores = model(vedio)
    loss = loss_function(tag_scores.view(1,25),labels.view(1))
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  print('[%d] loss: %.3f' % (epoch + 1,running_loss / len(train)))
  running_loss = 0.0\

epoch1
[1] loss: 0.454
epoch2
[2] loss: 0.284
epoch3
[3] loss: 0.177
epoch4
[4] loss: 0.145
epoch5
[5] loss: 0.122
epoch6
[6] loss: 0.096
epoch7
[7] loss: 0.084
epoch8
[8] loss: 0.072
epoch9
[9] loss: 0.064
epoch10
[10] loss: 0.055
epoch11
[11] loss: 0.050
epoch12
[12] loss: 0.043
epoch13
[13] loss: 0.042
epoch14
[14] loss: 0.039
epoch15
[15] loss: 0.037
epoch16
[16] loss: 0.031
epoch17
[17] loss: 0.028
epoch18
[18] loss: 0.026
epoch19
[19] loss: 0.023
epoch20
[20] loss: 0.022
epoch21
[21] loss: 0.020
epoch22
[22] loss: 0.019
epoch23
[23] loss: 0.017
epoch24
[24] loss: 0.015
epoch25
[25] loss: 0.014
epoch26
[26] loss: 0.012
epoch27
[27] loss: 0.011
epoch28
[28] loss: 0.010
epoch29
[29] loss: 0.009
epoch30
[30] loss: 0.008
epoch31
[31] loss: 0.007
epoch32
[32] loss: 0.006
epoch33
[33] loss: 0.006
epoch34
[34] loss: 0.005
epoch35
[35] loss: 0.006
epoch36
[36] loss: 0.005
epoch37
[37] loss: 0.005
epoch38
[38] loss: 0.004
epoch39
[39] loss: 0.004
epoch40
[40] loss: 0.004
epoch41
[41] loss:

---
---
## **Problem 3.** Evaluation

* ##### **Print the train and test accuracy of your model** 

In [133]:
# \*write your codes for evaluation (You can use multiple cells, this is just a place holder)
# Don't hardcode the train and test accuracy
from time import time
time1 = time()

correct = 0
total = 0
with torch.no_grad():
  for video,label in train:
    output = model(video)
    predicted = np.argmax(output.numpy())
    total += 1
    correct += (predicted == label).sum().item()
print('LSTM network training accuracy is %2.3f :' %(100 * correct / total)+'%' )

correct = 0
total = 0
with torch.no_grad():
  for video,label in test:
    output = model(video)
    predicted = np.argmax(output.numpy())
    total += 1
    correct += (predicted == label).sum().item()
print('LSTM network test accuracy is %2.3f :' %(100 * correct / total)+'%' )
time2 = time()
print("total "+str(time2-time1) + "seconds")

LSTM network training accuracy is 99.958 :%
LSTM network test accuracy is 82.229 :%
total 9.868932962417603seconds


* ##### **Print the train and test and test accuracy of SVM** 

In [143]:
# Don't hardcode the train and test accuracy
from sklearn.svm import LinearSVC
svc_train = []
for data,label in train:
  svc_train.append(data.view(1,-1))

svc_train = torch.cat(svc_train).numpy()
svc_model = LinearSVC(C=3)
svc_model.fit(svc_train,trainLabel)

LinearSVC(C=3, class_weight=None, dual=True, fit_intercept=True,
          intercept_scaling=1, loss='squared_hinge', max_iter=1000,
          multi_class='ovr', penalty='l2', random_state=None, tol=0.0001,
          verbose=0)

In [145]:
svc_test = []
for data,label in test:
  svc_test.append(data.view(1,-1))
svc_test = torch.cat(svc_test).numpy()

svctrain_result = svc_model.predict(svc_train)
train_accuracy = sum(svctrain_result == trainLabel)/len(svc_train)
svctest_result = svc_model.predict(svc_test)
test_accuracy = sum(svctest_result == testLabel)/len(svc_test)
print('SVM training accuracy is %2.3f ' %(100.00*train_accuracy)+'%')
print('SVM test accuracy is %2.3f ' %(100.00*test_accuracy)+'%')

SVM training accuracy is 100.000 %
SVM test accuracy is 83.701 %


## **Problem 4.** Report

In this homework, I learn to use LSTM to classify videos. The final accuracy of my LSTM network is 82%, the final accuracy of my SVM is 83%.

First, we can divide each video into 25 images to represent the video. In order to use VGG16 to represent the images, we need to resize each image from 256\*340 to 224\*224. Consider the main content is in the center of videos, we can crop 4 corner images and 1 center image from the raw image. Then we get the mean of these five 224\*224 images to get a 224\*224 image. We use 4096*25 matrix to represent a video.

Due to it takes a long time to extract all the videos, I save these data as '.mat' files into the google colab. This part is very important. It takes me a whole day to extract and save data. I think this is a little bit unreasonable since there are too much data we need to handle. If we do not extract correct data, it will be hard to train and it also will take more time to extract data again. I use same external codes from Internet to extract and save mat files and read these mat files in order to make sure I do these jobs correctly in one time.

Then I define my LSTM network sturcture. It has three parameters which are feature_dim,hidden_size and action_size. Feature_dim means the dim of feature. In this problem, dim is 4096. Action_size means the types of actions. In this problem, the number of types is 25. Hidden_size means the rnn hidden unit. I have tried 32,64 and 128. When hidden_size = 32, the accuracy of test set is about 48%. When hidden_size=64, the accuracy is 68%. When hidden_size=128, the accuracy is 82%. According to this, when hidden_size increases, the accuracy also increase. After the LSTM, I use a Linear network to output which the video belongs to.

The structure of LSTM is shown below.

LSTM(

  (lstm): LSTM(4096, 128)

  (out): Linear(in_features=128, out_features=25, bias=True)

)

I used to return the tag_score in the forward function like official document. I tried to use the log_softmax() and sigmoid() function. However, the loss doesn't change and just gets about 4% accuracy when testing. So I return tag_space when doing forward function. In this way, the loss successfully decrease.

When choosing the optimizer, I have tried the SGD optimizer and Adam optimizer.
Unlike the former homework in which Adam optimizer perform better, in this homework, Adam optimizer can not optimize the parameters and the accuracy about Adam optimizer is about 4%. So I use SGD optimizer. I also tried to change the lr of SGD optimizer. When lr = 0.1, the accuracy is 45%. And when lr=0.01, the accuracy is 82%.

After defining the network, I start to train the LSTM network. There is a trick that, we need to let the number of label minus 1 when extracting the data. Otherwise it will cause bug when training because the network output is from 0-24 and the label in the txt file is 1-25. Set epoch = 20 can get 40% accuracy. Set epoch = 40 can get 76% accuracy which is above the baseline of the requirement. Set epoch = 60 can get 82% accuracy. In addition, the training time is not too long.

Finally, I transfer the data into a long vector to train a SVM. Change C parameter doesn't change the result too much. When C =1 and C=3, the accuracy are both 83%.


## **Bonus**


* ##### **Print the size of your training and test data**

In [0]:
# Don't hardcode the shape of train and test data
print('Shape of training data is :', )
print('Shape of test/validation data is :', )

* ##### **Modelling and evaluation**

In [0]:
#Write your code for modelling and evaluation

## Submission
---
**Runnable source code in ipynb file and a pdf report are required**.

The report should be of 3 to 4 pages describing what you have done and learned in this homework and report performance of your model. If you have tried multiple methods, please compare your results. If you are using any external code, please cite it in your report. Note that this homework is designed to help you explore and get familiar with the techniques. The final grading will be largely based on your prediction accuracy and the different methods you tried (different architectures and parameters).

Please indicate clearly in your report what model you have tried, what techniques you applied to improve the performance and report their accuracies. The report should be concise and include the highlights of your efforts.
The naming convention for report is **Surname_Givenname_SBUID_report*.pdf**

When submitting your .zip file through blackboard, please
-- name your .zip file as **Surname_Givenname_SBUID_hw*.zip**.

This zip file should include:
```
Surname_Givenname_SBUID_hw*
        |---Surname_Givenname_SBUID_hw*.ipynb
        |---Surname_Givenname_SBUID_hw*.pdf
        |---Surname_Givenname_SBUID_report*.pdf
```

For instance, student Michael Jordan should submit a zip file named "Jordan_Michael_111134567_hw5.zip" for homework5 in this structure:
```
Jordan_Michael_111134567_hw5
        |---Jordan_Michael_111134567_hw5.ipynb
        |---Jordan_Michael_111134567_hw5.pdf
        |---Jordan_Michael_111134567_report*.pdf
```

The **Surname_Givenname_SBUID_hw*.pdf** should include a **google shared link**. To generate the **google shared link**, first create a folder named **Surname_Givenname_SBUID_hw*** in your Google Drive with your Stony Brook account. 

Then right click this folder, click ***Get shareable link***, in the People textfield, enter two TA's emails: ***bo.cao.1@stonybrook.edu*** and ***sayontan.ghosh@stonybrook.edu***. Make sure that TAs who have the link **can edit**, ***not just*** **can view**, and also **uncheck** the **Notify people** box.

Colab has a good feature of version control, you should take advantage of this to save your work properly. However, the timestamp of the submission made in blackboard is the only one that we consider for grading. To be more specific, we will only grade the version of your code right before the timestamp of the submission made in blackboard. 

You are encouraged to post and answer questions on Piazza. Based on the amount of email that we have received in past years, there might be dealys in replying to personal emails. Please ask questions on Piazza and send emails only for personal issues.

Be aware that your code will undergo plagiarism check both vertically and horizontally. Please do your own work.